In [ ]:
!pip install kaggle-environments --upgrade

In [ ]:
%%writefile submission.py
import random

last = -1
last_reward = 0
bandits = {i:{'steps': 0, 'decay': 1.0, 'reward':0.0, 'opponent_steps': 0} for i in range(100)}

def the1owl_agent(observation, configuration):
    global last, last_reward, bandits
    #Goal: Exploration/Extrapolation trade off
    bc = 100 #configuration['banditCount']

    olast = -1
    if observation['step'] > 0:
        bandits[last]['reward'] += observation['reward'] - last_reward
        for l in observation['lastActions']:
            bandits[l]['steps'] += 1
            bandits[l]['decay'] -= 0.03
            if l != last:
                bandits[l]['opponent_steps'] += 1
                olast = l
     
    #https://www.kaggle.com/c/santa-2020/discussion/202453 #Clarifaction that +.03% removed
    for i in range(bc):
        if i not in observation['lastActions']:
            bandits[i]['decay'] += 0.02 # should be +.03 but lowering for now
            if bandits[i]['decay'] >= 1.0:
                bandits[i]['decay'] = 1.0
    
    m = max([bandits[i]['decay'] for i in range(bc)])
    cbandits = [i for i in range(bc) if bandits[i]['decay'] >= m]
    
    if observation['step'] % 10 == 0:
        m = max([bandits[i]['opponent_steps'] for i in cbandits]) #competitive/FOMO
        cbandits = [i for i in cbandits if bandits[i]['opponent_steps'] >= m]
    
    if observation['step'] % 20 == 0:
        m = max([bandits[i]['reward'] for i in cbandits]) #greedy
        cbandits = [i for i in cbandits if bandits[i]['reward'] >= m]
    
    action = random.choice(cbandits)
    
    last_reward = observation['reward']
    if observation['step'] % 50 == 0:
        action = random.randrange(bc)
        
    if observation['step'] % 3 == 0: #Follow the leader
        if olast != -1:
            action = olast
            
    last = action
    return action

In [ ]:
from kaggle_environments import make
from IPython.core.display import HTML
#https://github.com/Kaggle/kaggle-environments/tree/master/kaggle_environments/envs/mab
env = make("mab", debug=True)
o = env.run(['submission.py', '../input/santa-2020/submission.py']) 
#HTML(env.render(mode="ipython", width=500, height=450, header=False))
print(o[1999][0]['reward'], o[1999][1]['reward'])
o = env.run(['submission.py', 'round_robin'])
print(o[1999][0]['reward'], o[1999][1]['reward'])